In [1]:
data = open('datawww.txt',encoding='utf-8').read()

In [2]:
corpus = data.lower().split("\n")

In [3]:
len(corpus)

3258

In [3]:
corpus

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:',
 'but thou, contracted to thine own bright eyes,',
 "feed'st thy light'st flame with self-substantial fuel,",
 'making a famine where abundance lies,',
 'thyself thy foe, to thy sweet self too cruel.',
 "thou that art now the world's fresh ornament",
 'and only herald to the gaudy spring,',
 'within thine own bud buriest thy content',
 'and, tender churl, makest waste in niggarding.',
 'pity the world, or else this glutton be,',
 "to eat the world's due, by the grave and thee.",
 'when forty winters shall beseige thy brow,',
 "and dig deep trenches in thy beauty's field,",
 "thy youth's proud livery, so gazed on now,",
 "will be a tatter'd weed, of small worth held:",
 "then being ask'd where all thy beauty lies,",
 'where all the treasure of thy lusty days,',
 'to say, within thine own deep-sunken eyes,',

In [5]:
for i in range(len(corpus)):
    corpus[i]=corpus[i].lstrip()

In [6]:
corpus

['trăm năm trong cõi người ta',
 'chữ tài chữ mệnh khéo là ghét nhau',
 'trải qua một cuộc bể dâu',
 'những điều trông thấy mà đau đớn lòng',
 'lạ gì bỉ sắc tư phong',
 'trời xanh quen thói má hồng đánh ghen',
 'cảo thơm lần giở trước đèn',
 'phong tình có lục còn truyền sử xanh',
 'rằng năm gia tĩnh triều minh',
 'bốn phương phẳng lặng hai kinh vững vàng',
 'có nhà viên ngoại họ vương',
 'gia tư nghĩ cũng thường thường bực trung',
 'một trai con thứ rốt lòng',
 'vương quan là chữ nối dòng nho gia',
 'đầu lòng hai ả tố nga',
 'thúy kiều là chị em là thúy vân',
 'mai cốt cách tuyết tinh thần',
 'một người một vẻ mười phân vẹn mười',
 'vân xem trang trọng khác vời',
 'khuôn trăng đầy đặn nét ngài nở nang',
 'hoa cười ngọc thốt đoan trang',
 'mây thua nước tóc tuyết nhường màu da',
 'kiều càng sắc sảo mặn mà',
 'so bề tài sắc lại là phần hơn',
 'làn thu thủy nét xuân sơn',
 'hoa ghen thua thắm liễu hờn kém xanh',
 'một hai nghiêng nước nghiêng thành',
 'sắc đành đòi một tài đành họa hai',

## Yêu cầu: Sinh từ tiếp khi gõ bất cứ một câu nào có chiều dài bất kỳ.

In [4]:
# despite of wrinkles -> this thy golden time to heart's sight ' must '

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)+1

# Create Input Sequence

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0] # chuyen cau thanh chuoi so
    for i in range (1,len(token_list)):
        n_gram_sequence = token_list[:i+1]
        print(n_gram_sequence)
        input_sequences.append(n_gram_sequence)

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

AttributeError: module 'numpy' has no attribute 'integer'

In [14]:
tokenizer.word_index['from']

34

### 1. Xử lý dữ liệu. Chia features, label

In [41]:
import numpy as np

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))

predictors,label = input_sequences[:,:-1],input_sequences[:,-1]

In [42]:
from tensorflow.keras.utils import to_categorical

label = to_categorical(label,num_classes=total_words)

In [39]:
label

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [25]:
max_sequence_len

11

In [20]:
predictors

array([[   0,    0,    0, ...,    0,    0,   34],
       [   0,    0,    0, ...,    0,   34,  417],
       [   0,    0,    0, ...,   34,  417,  877],
       ...,
       [   0,    0,    0, ..., 3209,  493,  493],
       [   0,    0,    0, ...,  493,  493, 3210],
       [   0,    0,    0, ...,  493, 3210,   15]], dtype=int32)

### 2. Xây dựng model
#### Yêu cầu độ chính xác: 80%

In [43]:
# Một lớp Embedding
# Một lớp Bidirectional với cell là LSTM
# Một lớp Dropout
# Một lớp LSTM
# 2 lớp Dense nhận diện từ tiếp theo 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense,Bidirectional,LSTM,Dropout


model = Sequential()
model.add(Embedding(total_words,100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150,return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2,activation='relu'))
model.add(Dense(total_words,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [44]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 10, 100)           321100    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 10, 300)           301200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 300)           0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_2 (Dense)              (None, 1605)              162105    
_________________________________________________________________
dense_3 (Dense)              (None, 3211)              5156866   
Total params: 6,101,671
Trainable params: 6,101,671
Non-trainable params: 0
____________________________________________

In [ ]:
test_seq = 'despite of wrinkles'

next_words = 10

for _ in range (next_words)
    token_list=tokenizer.texts_to_sequences([test_seq])[0]
    token_list=pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted = model.predict_classes(token_list,verbose=0)
    output_word=""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
        test_seq += " " + output_word
    print(test_seq)

In [45]:
history=model.fit(predictors,label,epochs=100,verbose=1)

Epoch 1/100
484/484 [==============================] - 31s 55ms/step - loss: 7.0003 - acc: 0.0217
Epoch 2/100
484/484 [==============================] - 27s 55ms/step - loss: 6.4459 - acc: 0.0286
Epoch 3/100
484/484 [==============================] - 28s 58ms/step - loss: 6.2379 - acc: 0.0363
Epoch 4/100
484/484 [==============================] - 32s 66ms/step - loss: 5.9941 - acc: 0.0455
Epoch 5/100
484/484 [==============================] - 27s 55ms/step - loss: 5.7474 - acc: 0.0541
Epoch 6/100
484/484 [==============================] - 29s 59ms/step - loss: 5.4686 - acc: 0.0669
Epoch 7/100
484/484 [==============================] - 27s 56ms/step - loss: 5.1917 - acc: 0.0799
Epoch 8/100
484/484 [==============================] - 27s 56ms/step - loss: 4.9607 - acc: 0.0866
Epoch 9/100
484/484 [==============================] - 27s 55ms/step - loss: 4.6837 - acc: 0.1060
Epoch 10/100
484/484 [==============================] - 26s 55ms/step - loss: 4.4128 - acc: 0.1228
Epoch 11/100
484/48

In [54]:
model.save('Shakespeare’s Sonnet 1.h5')

### 3. Dự đoán 10 từ tiếp theo

In [53]:
# sinh ra 10 từ tiếp theo sau test_seq
# despite of wrinkles this thy golden time to heart's sight ' must '

test_seq = 'despite of wrinkles'

next_words = 10

for _ in range (next_words):
    token_list=tokenizer.texts_to_sequences([test_seq])[0]
    token_list= pad_sequences([token_list],maxlen=max_sequence_len-1,padding='pre')
    predicted = model.predict_classes(token_list,verbose=0)
    output_word=""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    test_seq += " " + output_word
print(test_seq)

despite of wrinkles this thy golden time still shine new brought in other
